In [46]:
import datetime
from datetime import date
from datetime import datetime
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
# import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 100


In [3]:
#Getting Player Info Files
# Sravan's paths
# player_info_england = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
# player_info_italy = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
# player_info_spain = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
# player_info_france = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
# player_info_germany = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')


# Renzo's paths
player_info_england = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv')
player_info_italy = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv')
player_info_spain = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv')
player_info_france = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv')
player_info_germany = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv')


In [4]:
# Load TM Profile Datasets

df_1 = pd.read_csv('player_profile_df_1.csv')
df_2 = pd.read_csv('player_profile_df_2.csv')
df_3 = pd.read_csv('player_profile_df_3.csv')

tm_profile_df = pd.concat([df_1, df_2])
tm_profile_df = pd.concat([tm_profile_df, df_3])

In [5]:
player_inf_lst = [player_info_england, player_info_italy, player_info_spain, player_info_france, player_info_germany]
player_info_df = pd.concat(player_inf_lst)
player_info_df.shape

(35827, 15)

In [6]:
# Cleaning repeated players

player_info_df_nodups = player_info_df.drop_duplicates()
print(len(player_info_df))
print(len(player_info_df_nodups))

35827
10720


In [7]:
#Loading Player Injuries

# Sravan's Path
# player_injuries_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')

# Renzo's Path
player_injuries_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_df.csv')

len(player_injuries_df)

55216

In [8]:
print(player_injuries_df.columns)
print(player_info_df_nodups.columns)

Index(['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId',
       'name'],
      dtype='object')
Index(['FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality', 'Photo',
       'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram'],
      dtype='object')


In [59]:
# Load TMID to FBRefID reference table

# Sravan's Path
# fbref_to_tm_df = pd.read_csv('/Users/srav88/My Drive/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv', encoding='latin-1')

# Renzo's Path
fbref_to_tm_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv')

In [60]:
# Stripping FBRefID and TMID from other information and creating individual columns for them

# fbref_to_tm_df['FBRefID'] = fbref_to_tm_df['UrlFBref'].apply(lambda player: player[(player.find("players/") + len("players/")):(player.find("/matchlogs"))])

fbref_to_tm_df['FBRefID'] = fbref_to_tm_df['UrlFBref'].str.split('/').str[5]
fbref_to_tm_df['TMID'] = fbref_to_tm_df['UrlTmarkt'].str.split('/').str[6]

fbref_to_tm_df.head()


,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID
0,Aaron Connolly,https://fbref.com/en/players/27c01749/Aaron-Co...,https://www.transfermarkt.com/aaron-connolly/p...,Centre-Forward,27c01749,434207
1,Aaron Cresswell,https://fbref.com/en/players/4f974391/Aaron-Cr...,https://www.transfermarkt.com/aaron-cresswell/...,Left-Back,4f974391,92571
2,Aarón Escandell,https://fbref.com/en/players/67669ce7/Aaron-Es...,https://www.transfermarkt.com/aaron-escandell/...,Goalkeeper,67669ce7,284430
3,Aaron Herzog,https://fbref.com/en/players/565c3fe4/Aaron-He...,https://www.transfermarkt.com/aaron-herzog/pro...,Attacking Midfield,565c3fe4,276566
4,Aaron Hickey,https://fbref.com/en/players/1780bb4a/Aaron-Hi...,https://www.transfermarkt.com/aaron-hickey/pro...,Left-Back,1780bb4a,591949


In [61]:
# converting TMId to string in player_injuries_df

player_injuries_df['TMId'] = player_injuries_df['TMId'].astype(str)
# player_injuries_df.head()

In [62]:
# Merging on intersection of player_injuries_df and fbref_to_tm_df on columns TMId and TMID respectively

player_injuries_df_2 = pd.merge(left=player_injuries_df, right=fbref_to_tm_df, left_on='TMId', right_on='TMID', how='inner')

# player_injuries_df_2.head()

In [63]:
# Determining number of unique player in our data

len(player_injuries_df_2['name'].unique())

4614

In [64]:
# removing duplicates from player_info_df

player_info_df = player_info_df.drop_duplicates()

# player_info_df.head()

In [65]:
# Merging Player Injuries with FBRef Profiles

player_injuries_info_df = pd.merge(left=player_injuries_df_2, right=player_info_df, left_on='FBRefID', right_on='FBRefId', how='inner')

len(player_injuries_info_df)

32584

In [66]:
# finding number of unique players
 
len(player_injuries_info_df['name'].unique())

4586

In [67]:
# Merge with TM Profile information

tm_profile_df['TMId'] = tm_profile_df['TMId'].astype(str)
player_injuries_profile_final = pd.merge(left=player_injuries_info_df, right=tm_profile_df, left_on='TMId', right_on='TMId', how='inner')
player_injuries_profile_final.head()

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:
0,18/19,Hamstring Injury,"Sep 29, 2018","Oct 27, 2018",28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16/17,Calf Strain,"Mar 18, 2017","Apr 17, 2017",30 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15/16,Calf Injury,"Mar 3, 2016","Mar 31, 2016",28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14/15,Neck Injury,"May 11, 2015","May 25, 2015",14 days,2,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13/14,Shoulder Injury,"Apr 24, 2014","Jul 17, 2014",84 days,4,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermark

In [68]:
# Creating new columns with features

player_injuries_profile_final = player_injuries_profile_final[player_injuries_profile_final['from'] != '-']
player_injuries_profile_final = player_injuries_profile_final[player_injuries_profile_final['until'] != '-']
player_injuries_profile_final['injury_year'] = player_injuries_profile_final['from'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').year)
player_injuries_profile_final['injury_week'] = player_injuries_profile_final['from'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%V'))
player_injuries_profile_final['release_week'] = player_injuries_profile_final['until'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%V'))

In [88]:
# Exploding Dataframe to weekly basis

# player_injuries_profile_final['from']=player_injuries_profile_final.apply(lambda x : pd.date_range(start=x['from'], end=x['until'], freq='W') ,axis=1)
# player_injuries_profile_final = player_injuries_profile_final.explode('from')
# player_injuries_profile_final['until']=player_injuries_profile_final['from']+pd.tseries.offsets.MonthEnd()

# player_injuries_profile_final.head(50)

def ran(row):
    return list(range(row['injury_week'], row['release_week']))

player_injuries_profile_final['injury_week'] = player_injuries_profile_final['injury_week'].astype(int)
player_injuries_profile_final['release_week'] = player_injuries_profile_final['release_week'].astype(int)
player_injuries_profile_final['current_week'] = player_injuries_profile_final.apply(ran, axis = 1)

player_injuries_profile_final = player_injuries_profile_final.explode('current_week')


In [87]:
player_injuries_profile_final.head(50)

,Season,Injury,from,until,Days,Games missed,TMId,name_x,PlayerFBref,UrlFBref,UrlTmarkt,TmPos,FBRefID,TMID,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height,Weight,Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram,Place of birth:,Citizenship:,Height:,Position:,Contract expires:,Former International:,Caps/Goals:,League level:,Joined:,Contract until:,name_y,Date of birth,Last club:,Most games for:,Unknown since:,Date of birth:,Age:,Foot:,Player agent:,Current club:,Agent:,National player:,Name in home country:,Outfitter:,Current international:,Social-Media:,Full name:,On loan from:,Contract there expires:,Contract option:,Retired since:,club:,Without Club since:,2nd club:,country:,Company:,Date of death:,Career break since:,3nd club:,Ban since:,injury_year,injury_week,release_week,week_range
0,18/19,Hamstring Injury,2018-09-30,2018-10-31,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,39
0,18/19,Hamstring Injury,2018-09-30,2018-10-31,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,40
0,18/19,Hamstring Injury,2018-09-30,2018-10-31,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,41
0,18/19,Hamstring Injury,2018-09-30,2018-10-31,28 days,5,5658,Petr Cech,Petr Čech,https://fbref.com/en/players/71672fa0/Petr-Cech,https://www.transfermarkt.com/petr-cech/profil...,Goalkeeper,71672fa0,5658,71672fa0,Petr Čech,Petr Čech,Czech Republic,https://fbref.com/req/202005121/images/headsho...,"May 20, 1982",196.0,90.0,GK,LEFT,75.0,4x Premier League Champion||2011-12 Champions ...,https://fbref.com/en/players/71672fa0/Petr-Cech,https://twitter.com/PetrCech,https://instagram.com/petrcech/,Plzeň,Czech Republic,"1,96 m",Goalkeeper,-,Czechia,124/0,NaN,"Jul 1, 2019",NaN,Petr Cech,NaN,Arsenal,Chelsea,NaN,"May 20, 1982",39.0,left,SPORT INVEST,Retired,NaN,NaN,Petr Čech,Puma,NaN,NaN,NaN,NaN,NaN,NaN,"Jul 1, 2019",Chelsea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,39,43,42
0,18/19,Hamstring Injury,2018-10-07,2018-10-31,28 days,5,5658,Petr Cech,P

In [75]:
player_injuries_profile_final['injury_week'].value_counts(dropna = False)

39    4511
43    4350
38    4281
09    4050
49    3999
44    3965
37    3964
34    3854
45    3821
42    3804
05    3747
41    3743
40    3697
47    3637
48    3637
35    3630
11    3584
33    3555
03    3437
50    3430
07    3413
06    3383
02    3368
36    3360
10    3320
32    3313
08    3282
01    3194
14    3137
04    3072
31    3063
16    3059
51    3013
13    2961
29    2903
17    2879
30    2841
18    2772
46    2676
15    2646
12    2365
19    2314
27    2235
28    2012
52    1632
21    1620
26    1560
20    1465
23    1307
22    1268
24    1137
25    1101
53     995
Name: injury_week, dtype: int64

In [33]:
# Loading Match Dataframes

# Renzo's Path
total_match_logs_df = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/consolidated_df.csv')

/Users/renzomaldonado/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19,20,21,23,24,25,26,27,28,29,30,31,32,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [37]:
print(total_match_logs_df.columns)
print(player_injuries_profile_final.columns)

Index(['Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent',
       'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY',
       'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks', 'xG', 'npxG', 'xA',
       'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'Prog', 'Carries', 'Prog.1', 'Succ',
       'Att.1', 'name', 'FBRefID', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'OG',
       'PKwon', 'PKcon'],
      dtype='object')
Index(['Season', 'Injury', 'from', 'until', 'Days', 'Games missed', 'TMId',
       'name_x', 'PlayerFBref', 'UrlFBref', 'UrlTmarkt', 'TmPos', 'FBRefID',
       'TMID', 'FBRefId', 'PlayerName', 'PlayerFullName', 'Nationality',
       'Photo', 'Birth', 'Height', 'Weight', 'Position', 'Foot', 'FootAbility',
       'InternationalReputation', 'PlayerUrl', 'Twitter', 'Instagram',
       'Place of birth:', 'Citizenship:', 'Height:', 'Position:',
       'Contract expires:', 'Former International:', 'Caps/Goals:',
       'League level:', 'Joined:', 'Contract u

In [32]:
# Saving player_injuries_profile_final to Google Drive

# Renzo's path

player_injuries_profile_final.to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_injuries_profile_final.csv', index=False)